In [2]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob

In [14]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
def preprocesar_texto(texto):
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^\w\s]', '', texto)
    
    # Convertir el texto a minúsculas
    texto = texto.lower()
    
    # Tokenización: dividir el texto en palabras
    palabras = word_tokenize(texto)
    
    # Eliminar palabras de parada (stop words)
    stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tu conjunto de datos
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]
    
    # Reunir las palabras filtradas en un solo texto
    texto_preprocesado = ' '.join(palabras_filtradas)
    
    return texto_preprocesado

In [16]:
preprocesar_texto("A great game! It's amazing and very funny")

'great game amazing funny'

In [5]:
user_reviews = pd.read_json("./data/processed_json/reviews_desanidados.json")

In [7]:
user_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [17]:
user_reviews['review'] = user_reviews['review'].apply(preprocesar_texto)

In [9]:
# Función para analizar el sentimiento
def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity > 0:
        return 2  # Positivo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 0  # Negativo

In [19]:
# Aplica el análisis de sentimiento
user_reviews['sentiment_analysis'] = user_reviews['review'].apply(analizar_sentimiento)

In [21]:
user_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,sentiment_analysis
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,simple yet great replayability opinion zombie ...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,unique worth playthrough,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,great atmosphere gunplay bit chunky times end ...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,know think see title barbie dreamhouse party i...,js41637,http://steamcommunity.com/id/js41637,2
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,simple actually simple truck driving simulator...,js41637,http://steamcommunity.com/id/js41637,0


In [22]:
user_reviews['sentiment_analysis'].fillna(1, inplace=True)

In [23]:
user_reviews.to_csv("./data/processed_json/reviews_with_sentiment_analysis.csv", index= False)